Here is imported the necessary libreries and objects from agents.py

(clear_output is used to clear the current output and see the program more like an animation, time is used to pause between output and output)

In [1]:
import time
from IPython.display import clear_output
from agents import *
step = 0

In this section we create the class that defines our agent. First we define the methods that set the movements for the agent, then we create other three methods to know the state of the agent

In [2]:
class MAgent(Agent):
    def __init__(self, program=None):
        super().__init__(program)
        self.things = []
        self.last_action = "none"
        self.sign = "a "

    # Movements

    def moveRight(self):
        self.location = (self.location[0] + 1, self.location[1])

    def moveLeft(self):
        self.location = (self.location[0] - 1, self.location[1])

    def moveDown(self):
        self.location = (self.location[0], self.location[1] + 1)

    def moveUp(self):
        self.location = (self.location[0], self.location[1] - 1)

    def noOp(self):
        pass

    # Agent State

    def is_all_seen(self):
        result = True
        for y in range(6):
            for x in range(6):
                if self.thing_at((x, y)) == "? ":
                    return False
        return result

    def remove_percepted_thing(self, location):
        for thing in self.things:
            if thing.location == location:
                self.things.remove(thing)

    def add_perecpted_things(self, things):
        for thing in things:
            self.things.append(thing)

    def thing_at(self, location):
        for thing in self.things:
            if thing.location == location:
                return thing.sign
        return "? "

    def show_state(self):
        print('\n' + " Agent's internal state: " + '\n')
        print("  0 1 2 3 4 5 ")
        for y in range(6):
            print(y, end=' ')
            for x in range(6):
                print(self.thing_at((x, y)), end='')
            print()

In this section we create the classes of the elements of the enviroment and its attributes

In [3]:
class AThing(Thing):
    def __init__(self,value,sign,location=(0,0)):
        self.location = location
        self.value = value
        self.sign = sign

class Nothing(AThing):
    def __init__(self,location=(0,0)):
        super().__init__(-1,"- ",location)

class Outside(AThing):
    def __init__(self,location=(0,0)):
        super().__init__(-10,"o ",location)

class Wall(AThing):
    def __init__(self):
        super().__init__(-5,"w ")

# Treasures

class Treasure(AThing):
    def __init__(self,value,sign):
        super().__init__(value,sign)

class Treasure_gold(Treasure):
    def __init__(self):
        super().__init__(20,"g ")

class Treasure_diamond(Treasure):
    def __init__(self):
        super().__init__(50,"d ")

# Traps

class Trap(AThing):
    def __init__(self,value,sign):
        super().__init__(value,sign)

class Snake_trap(AThing):
    def __init__(self,):
        super().__init__(-20,"s ")

class Hole_trap(AThing):
    def __init__(self):
        super().__init__(-15,"h ")

The class MEnv(Enviroment) contents the methods that creates and modifies the environment attributes.

* The method percept is used to know what the agent has around it.
* The method execute_action has the function of moves the agent, update and print its location and performance.
* The method show_grid displays the map.

In [4]:
class MEnv(Environment):

    # Things displayed (different from thing_at, because this includes the agent)
    def disp_thing_at(self,agent,location):
        if agent.location == location:
            return agent
        elif location[0] < 0 or location[1] < 0 or location[0] > 5 or location[1] > 5:
            return Outside(location)
        elif self.list_things_at(location):
            return self.list_things_at(location)[0]
        else:
            return Nothing(location)

    # Different from disp_thing_at, because this excludes the agent)
    def thing_at(self, agent, location):
        if location[0] < 0 or location[1] < 0 or location[0] > 5 or location[1] > 5:
            return Outside(location)
        elif self.list_things_at(location):
            # This returns the thing that isn't the agent and if it doesn't found anything returns nothing
            if agent in self.list_things_at(location):
                for t in self.list_things_at(location):
                    if not isinstance(t,MAgent):
                        return t
                return Nothing(location)
            else:
                return self.list_things_at(location)[0]
        else:
            return Nothing(location)

    def percept(self, agent):
        location = agent.location

        here = self.thing_at(agent, location)
        left = self.thing_at(agent, (location[0] - 1, location[1]))  # Left
        down = self.thing_at(agent, (location[0], location[1] + 1))  # Down
        right = self.thing_at(agent, (location[0] + 1, location[1]))  # Right
        up = self.thing_at(agent, (location[0], location[1] - 1))  # Up

        here.position = "nop"
        left.position = "left"
        down.position = "down"
        right.position = "right"
        up.position = "up"

        percepted_things = [here, left, down, up, right]

        print("<BEFORE THE ACTION>")
        self.show_grid(agent)
        agent.add_perecpted_things(percepted_things)
        agent.show_state()
        print("Agent performance: ",agent.performance)
        return {"agent": agent,"positions": percepted_things}


    def execute_action(self, agent, action):
        print("Action: ",action)
        if action == "nop":
            pass
        else:
            agent.performance -= 1
            if action == "right":
                agent.moveRight()
            elif action == "left":
                agent.moveLeft()
            elif action == "up":
                agent.moveUp()
            elif action == "down":
                agent.moveDown()
            elif action == "nop":
                agent.noOp

        location = agent.location
        thing = self.thing_at(agent,location)

        # Discriminate path already covered to prevent loops
        if isinstance(thing,Nothing):
            self.add_thing(Nothing(),location)
        thing.value = thing.value - 1

        if isinstance(thing, Treasure):
            self.delete_thing(thing)
            agent.remove_percepted_thing(thing.location)
            agent.performance += thing.value

        if isinstance(thing, Trap):
            agent.performance += thing.value

        agent.last_action = action
        print("<AFTER THE ACTION>")
        self.show_grid(agent)
        agent.show_state()
        print("Agent performance: ",agent.performance)
        #print("----------------------------------------------------------------------")

        # Esto es para que no se borre el output y pueda verse más fluido
        global step
        step += 1
        print("Step: ",step)
        clear_output(wait=True)
        time.sleep(1)

    def show_grid(self,agent):
        print(" The environment content: " + '\n')
        print("  0 1 2 3 4 5 ")
        for y in range(6):
            print(y, end=' ')
            for x in range(6):
                print(self.disp_thing_at(agent,(x, y)).sign, end='')
            print()

In this section we design the program that the agent follows and create some useful functions.

* the method remove_invalid is used to avoid getting the agent outside of the map or moving inside a wall
* the method areEquals is used to known if the possible next movements has the same value in the performance
* the method program has the conditions that the agent use to do the next move

In [5]:
def remove_invalid(percepts):
    percs = []
    for x in percepts:
        if isinstance(x,Outside) or isinstance(x,Wall):
            pass
        else:
            percs.append(x)
    return percs

def areEquals(e):
    if e[0] == e[1] == e[2] == e[3]:
        return True
    else:
        return False

def program(percepts):

    print("<AGENT PROGRAM>")

    agent=percepts["agent"]
    location = agent.location

    percepted_things = percepts["positions"]
    print("Percept: ({},({},{},{},{},{}))".format(location, percepted_things[0].sign, percepted_things[1].sign, percepted_things[2].sign, percepted_things[3].sign, percepted_things[4].sign))

    validThings = remove_invalid(percepted_things)

    # If the agent has seen all the grid and there is no treasures left to collect
    if agent.is_all_seen() and (not any(isinstance(x, Treasure) for x in percepts["positions"])):
        return "nop"
    elif areEquals(validThings):
        best_choice = random.choice(validThings)
        return best_choice
    else:
        xx = max(validThings, key=lambda x: x.value).position
        best_choice = xx
        return best_choice

Finally, we create all instances, create the structure of the map, define the starting point of the agent and the number of steps.

In [8]:
e = MEnv()

#row 0
e.add_thing(Treasure_gold(), (0, 2))
#row 1
e.add_thing(Hole_trap(), (0, 1))
e.add_thing(Snake_trap(), (2, 1))
e.add_thing(Wall(), (3, 1))
#row 2
e.add_thing(Treasure_diamond(), (0, 3))
e.add_thing(Wall(), (3, 2))
e.add_thing(Treasure_diamond(), (5, 2))
#row 3
#e.add_thing(Wall(),(3,3))
e.add_thing(Wall(), (2, 3))
e.add_thing(Wall(), (1, 3))
#row 4
e.add_thing(Hole_trap(), (3, 4))
e.add_thing(Treasure_diamond(), (4, 4))
#row 5
e.add_thing(Snake_trap(), (0, 5))

a = MAgent(program)
#TraceAgent(a)
e.add_thing(a, (5, 5))
global step
step = 0
e.run(100)

<BEFORE THE ACTION>
 The environment content: 

  0 1 2 3 4 5 
0 - - - - - - 
1 h - s w - - 
2 - - - w - - 
3 - w w a - - 
4 - - - h - - 
5 s - - - - - 

 Agent's internal state: 

  0 1 2 3 4 5 
0 - - - - - - 
1 h - s w - - 
2 - - - w - - 
3 - w w - - - 
4 - - - h - - 
5 s - - - - - 
Agent performance:  138
<AGENT PROGRAM>
Percept: ((3, 3),(- ,w ,h ,w ,- ))
Action:  nop
<AFTER THE ACTION>
 The environment content: 

  0 1 2 3 4 5 
0 - - - - - - 
1 h - s w - - 
2 - - - w - - 
3 - w w a - - 
4 - - - h - - 
5 s - - - - - 

 Agent's internal state: 

  0 1 2 3 4 5 
0 - - - - - - 
1 h - s w - - 
2 - - - w - - 
3 - w w - - - 
4 - - - h - - 
5 s - - - - - 
Agent performance:  138
Step:  98
